# Projekt 2 - EDA
Online shoppers intention zawierają informację o aktywności użytkowników w sesji i czy w trakcie sesji użytkownik dokonał jakiegoś zakupu

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas_profiling
import copy
from sklearn.decomposition import PCA
import sklearn.metrics
from sklearn import manifold

## Podstawowe informacje
W celu pozyskania podstawowych informacji użyjemy narzędzia do zautomatyzowanej eksploracji danych **pandas_profiling**. 

In [ ]:
df=pd.read_csv("online_shoppers_intention.csv")
df=df.info()

Nie ma braków danych
Zmienna Month i VisitorsType (Returning, New, Other) typu object
Zmienna Weekend i Revenue typu bool.

Zmienne `Administrative`, `Informational`, `ProductRelated` przedstawiają ile stron danego typu odwiedził użytkownik

`Administrative_Duration`,  `Informational_Duration`, `ProductRelated_Duration` ile czasu w sumie użytkownik spędził na stronie danego typu

`BounceRates` - procent stron które użytkownik odwiedził bez żadnej dalszej interakcji

`SpecialDay` - jak blisko specjalnego dnia była sesja

In [ ]:
df=pd.read_csv("online_shoppers_intention.csv")
df=df.dropna()

In [ ]:
df.profile_report()

### Wnioski:
* dużo zer w zmiennych opisujących pobyt na konkretnych typach stron 
* Większość wizyt nie kończy się transakcją
* Przewaga przeglądarki 1. i 2.
* Czy jedna sesja = jedna witryna? Czym są zmienne opisujące strony


## Macierz korelacji

In [ ]:
nums=["Administrative", "Administrative_Duration", 
      "Informational", "Informational_Duration", 
      "ProductRelated", "ProductRelated_Duration",
     "BounceRates", "ExitRates", "PageValues",
     "SpecialDay", "VisitorType", "Revenue"]
df_hm=df[nums]
df_hm=df_hm.replace(False, 0)
df_hm=df_hm.replace(True, 1)
df_hm=df_hm.replace("New_Visitor", 0)
df_hm=df_hm.replace("Returning_Visitor", 1)
df_hm=df_hm.replace("Other", 0.5)   #jest ich stosunkowo mało więc to bez większego znaczenia 
df_hm=df_hm.replace("")
pd.to_numeric(df_hm["VisitorType"])
df_hm=df_hm
plt.figure(figsize=(10, 10))
sns.heatmap(df_hm.corr(), annot=True, annot_kws={'size': 11}, fmt='.2f')

### Wnioski/pytania do macierzy korelacji
* Ujemna korelacjia pomiędzy `SpecialDay` i wszystkimi zmiennymi dot. czasu spędzonego na stronach oraz zmiennej `Revenue` - zaskakujące
* Ujemna korelacja pomiędzy `ExitRates`  i `Revenue` - też nieintuicyjne
* Czym tak naprawdę jest `PageValues`
* Bardzo duża pomiędzy `ExitRates` i `BounceRates` - czy to ludzie którzy po odpaleniu strony od razu ją zamknęli?

In [ ]:
sns.set()
cols = ['Administrative', 'Informational',
       "ProductRelated"]
sns.pairplot(df[cols], size = 2.5)
plt.show();

In [ ]:
sns.set()
cols = ['ExitRates', 'BounceRates',
       "PageValues"]
sns.pairplot(df[cols], size = 2.5)
plt.show();

Dla ExitRates/BounceRates jedna obserwacja znajduje się w innej częsci trójkąta

## Proporcje w spędzonym czasie w zależności od zakupu

In [ ]:
inf_dur=df["Informational_Duration"].sum()
prod_dur=df["ProductRelated_Duration"].sum()
adm_dur=df["Administrative_Duration"].sum()
labels = ['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration']
sizes = [adm_dur, inf_dur, prod_dur]
colors=["khaki", "mediumseagreen", "skyblue"]
fig, axs=plt.subplots(ncols=1, nrows=3, figsize=(20,20))
axs[0].pie(sizes, labels=labels,  autopct='%1.1f%%', textprops={'fontsize': 14}, colors=colors)
axs[0].set_title("All users", fontsize=16)
X=df[df["Revenue"]==True]
inf_dur=X["Informational_Duration"].sum()
prod_dur=X["ProductRelated_Duration"].sum()
adm_dur=X["Administrative_Duration"].sum()
labels = ['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration']
sizes = [adm_dur, inf_dur, prod_dur]
colors=["khaki", "mediumseagreen", "skyblue"]
axs[1].pie(sizes, labels=labels,  autopct='%1.1f%%', textprops={'fontsize': 14}, colors=colors)
axs[1].set_title("Revenue == True", fontsize=16)
X=df[df["Revenue"]==False]
inf_dur=X["Informational_Duration"].sum()
prod_dur=X["ProductRelated_Duration"].sum()
adm_dur=X["Administrative_Duration"].sum()
labels = ['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration']
sizes = [adm_dur, inf_dur, prod_dur]
colors=["khaki", "mediumseagreen", "skyblue"]
axs[2].pie(sizes, labels=labels,  autopct='%1.1f%%', textprops={'fontsize': 14}, colors=colors)
axs[2].set_title("Revenue == False", fontsize=16)

## Długość pobytu na stronie a zakup

In [ ]:
X=df.copy()
plt.figure(figsize=(10, 10))
X["Time"]=df["Informational_Duration"]+df["ProductRelated_Duration"]+df["Administrative_Duration"]
sns.boxplot(data=X, x="Revenue", y="Time")
plt.ylim(-100, 30000)

## Rodzaje ruchu

In [ ]:
plt.figure(figsize=(10, 10))
X = pd.crosstab(df['TrafficType'], df['Revenue'])
X.reset_index(inplace=True)
plt.bar(X["TrafficType"], X[True]+X[False])
plt.title("Ilość wizyt poszczególnego typu (TrafficType)")

## Rodzaj ruchu a zakup

In [ ]:
plt.figure(figsize=(10, 10))
X = pd.crosstab(df['TrafficType'], df['Revenue'])
X.reset_index(inplace=True)
plt.bar(X["TrafficType"], X[True]/X[False])
plt.title("Procent wizyt zakończonych transakcją a rodzaj ruchu")

## Miesiące


In [ ]:
X=df.copy()
months = ["Jan", "Feb", "Mar", "Apr", "May", "June", 
          "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
X["Time"]=df["Informational_Duration"]+df["ProductRelated_Duration"]+df["Administrative_Duration"]
X['Month'] = pd.Categorical(X['Month'], categories=months, ordered=True)
X=X.groupby(["Month"]).agg({'Time':'sum'}).reset_index()
X = X.sort_values(by="Month")
plt.figure(figsize=(10, 10))
plt.bar(X["Month"], X["Time"])
plt.title("Łączny czas spędzony przez użytkowników podczas wszystkich sesji")

In [ ]:
X=df.copy()
X=X.replace(True, 1)
X=X.replace(False, 0)
months = ["Jan", "Feb", "Mar", "April", "May", "June", 
          "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
X['Month'] = pd.Categorical(X['Month'], categories=months, ordered=True)
X=X.groupby(["Month"]).agg({'Revenue':['sum', "count"]}).reset_index()
X = X.sort_values(by="Month")
X.columns=["Month", "sum", "count"]
plt.figure(figsize=(10, 10))
plt.bar(X["Month"], X["sum"]/X["count"])
plt.title("Procent wizyt zakończonych transakcją")




In [ ]:
df[(df["Month"]=="Feb") & (df["Revenue"]==True)]

## PCA

In [ ]:
nums=["Administrative_Duration", 
      "Informational_Duration", 
    "ProductRelated_Duration", "SpecialDay"]  #bierzemy zmienne opisujące  użytkowników a nie strony
X=df[nums]
X=(X-X.mean())/X.std()
pca=PCA(2)
pca.fit(X)
plt.figure(figsize=(10, 10))
X_pca=pca.transform(X)
sns.scatterplot(X_pca[:, 0], X_pca[:,1], hue=df["Revenue"]) 


### Wnioski:
Z powodu dużej ilości zer w naszych danych, PCA nie daje szczególnie dobrych rezultatów. Widoczne na wykresie grupki wynikają z wartości zmiennej "SpecialDay", która przyjmuje w końcu ograniczoną ilość wartości - zależną od dnia, nie użytkownika.

## Weekend

In [ ]:
df.Weekend.value_counts()

In [ ]:
plt.figure(figsize=(10, 10))
tmp=df[["Weekend", "Revenue"]].groupby(["Revenue", "Weekend"]).size().tolist()
plt.bar(["Weekend","not Weekend"], list(reversed(tmp[0:2])), label='Revenue - False', color="blue")
plt.bar(["Weekend","not Weekend"], list(reversed(tmp[2:4])), bottom=list(reversed(tmp[0:2])),
       label='Revenue - True', color="red")
plt.legend()
plt.title("Weekend x Revenue")
plt.show()

* Wiecej sesji odbyło się w ciagu tygodnia
* Revenue w zależnosci od weekendu rozkłada sie podobnie w obu przypadkach, większość stanowi Revenue = False

## Revenue vs  PageValues/ ExitRates/ BoinceRates

In [ ]:
plt.figure(figsize=(10, 10))
sns.violinplot(x = df['Revenue'], y = df['PageValues'])
plt.title('Revenue vs PageValues')

In [ ]:
plt.figure(figsize=(10, 10))
sns.violinplot(x = df['Revenue'], y = df['ExitRates'])
plt.title('Revenue vs ExitRates')

In [ ]:
plt.figure(figsize=(10, 10))
sns.violinplot(x = df['Revenue'], y = df['BounceRates'])
plt.title('Revenue vs BounceRates')

Dla  ExitRates i BounceRates  dla Revenue = False widać duża liczbę obserwacji o okolicy 2 i zaóważaliny wzrost w okolicy 0.20



## Month vs SpecialDay

In [ ]:
plt.figure(figsize=(10, 10))
sns.violinplot(x = df['Month'], y = df['SpecialDay'], inner = 'points')
plt.title('Month vs SpecialDay')

## VisitorType

In [ ]:
plt.figure(figsize=(10, 10))
sns.violinplot(x = df['VisitorType'], y = df['BounceRates'], inner = 'points')
plt.title('VisitorType vs BounceRates')

In [ ]:
plt.figure(figsize=(10, 10))
sns.violinplot(x = df['VisitorType'], y = df['ExitRates'], inner = 'points')
plt.title('VisitorType vs ExitRates')

New_Visitor dla ExitRates i BounceRates ma dużo obserwacji w okolicy 0. Dla Returning_Visitors instnieje wiele obcerwacji z wartościami pomiędzy 0 a 0.20.

In [ ]:
plt.figure(figsize=(10, 10))
sns.violinplot(x = df['VisitorType'], y = df['PageValues'], inner = 'points')
plt.title('VisitorType vs PageValues')

Najwyższe PageValues jest dla New_Visitor. tym razem dużo obserwacji bliskich 0 przypada Returning_Visitors